In [ ]:
'''
378300 / (300 * 22) = 57.31
378300 / (250 * 22) = 68.78
378300 / (200 * 22) = 85.97
378300 / (100 * 22) = 171.95
378300 / (80 * 22) = 214.94
378300 / (55 * 22) = 312.64
378300 / (42 * 22) = 409.41
'''

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.neighbors import NearestNeighbors

In [ ]:
# Leitura dos dados
data = pd.read_csv("../data_source/amostra_total.csv", sep=';')
data = data[["INDICE", "LATITUDE", "LONGITUDE", "LOGRADOURO", "NUMERO"]]

In [ ]:
# Definição de parâmetros
first_n_clusters = 42
n_of_days = 22

In [ ]:
# Clusterização inicial por leiturista
kmeans = MiniBatchKMeans(n_clusters=first_n_clusters, random_state=8081)
data['LEITURISTA'] = kmeans.fit_predict(data[['LATITUDE', 'LONGITUDE']])

In [ ]:
# Inicialização das colunas DIA e ROTA
data['DIA'] = -1
data['ROTA'] = -1

In [ ]:
# Processamento de cada cluster de leiturista
for leiturista in range(first_n_clusters):
    subcluster_data = data[data['LEITURISTA'] == leiturista]
    if len(subcluster_data) >= n_of_days:
        kmeans_22 = MiniBatchKMeans(n_clusters=n_of_days, random_state=8081)
        subcluster_data['DIA'] = kmeans_22.fit_predict(subcluster_data[['LATITUDE', 'LONGITUDE']])
    else:
        subcluster_data['DIA'] = np.arange(len(subcluster_data))
    
    # Atualização do dataframe principal
    data.loc[subcluster_data.index, 'DIA'] = subcluster_data['DIA']

In [ ]:
# Criação da coluna ROTA
data['ROTA'] = data.apply(lambda row: row['LEITURISTA'] * n_of_days + row['DIA'], axis=1)

In [ ]:
# Salvamento do resultado em CSV
data.to_csv(f'../cluster/{first_n_clusters}clusters.csv', index=False)

In [ ]:
# Função para plotar os clusters
def plot_clusters_sns(df, num_clusters, cluster_col, title, filename):
    plt.figure(figsize=(10, 6))
    palette = sns.color_palette("hsv", num_clusters)
    sns.scatterplot(data=df, x='LONGITUDE', y='LATITUDE', hue=cluster_col, palette=palette, s=50, legend=None)
    plt.title(title)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.savefig(filename)
    plt.close()

In [ ]:
# Plotagem dos clusters
plot_clusters_sns(data, first_n_clusters, 'LEITURISTA', f'{first_n_clusters} Clusters', f'../cluster/{first_n_clusters}_clusters.png')

In [ ]:
# Contagem do número de pontos por rota
rota_counts = data['ROTA'].value_counts()

In [ ]:
# Classificação das rotas
below_range = rota_counts[rota_counts < 350].count()
within_range = rota_counts[(rota_counts >= 350) & (rota_counts <= 450)].count()
above_range = rota_counts[rota_counts > 450].count()

In [ ]:

# Exibição dos resultados
print(f"Rotas abaixo de 350 pontos: {below_range}")
print(f"Rotas dentro do intervalo de 350-450 pontos: {within_range}")
print(f"Rotas acima de 450 pontos: {above_range}")

## Tentando balancear as rotas

In [ ]:
# Função para balancear as rotas iterativamente
def iterative_balance_routes(df, min_points=350, max_points=450, max_iterations=100):
    df_balanced = df.copy()
    iteration = 0
    
    while iteration < max_iterations:
        rota_counts = df_balanced['ROTA'].value_counts()
        below_routes = rota_counts[rota_counts < min_points].index
        above_routes = rota_counts[rota_counts > max_points].index
        
        if not below_routes.any() and not above_routes.any():
            break
        
        for route in below_routes:
            points_needed = min_points - len(df_balanced[df_balanced['ROTA'] == route])
            closest_points = df_balanced[df_balanced['ROTA'].isin(above_routes)][['LATITUDE', 'LONGITUDE']].to_numpy()
            if closest_points.size == 0:
                continue
            
            neigh = NearestNeighbors(n_neighbors=points_needed)
            neigh.fit(closest_points)
            nearest_points_idx = neigh.kneighbors(df_balanced[df_balanced['ROTA'] == route][['LATITUDE', 'LONGITUDE']].to_numpy(), return_distance=False)
            nearest_points = df_balanced.iloc[nearest_points_idx.flatten()]
            points_to_transfer = min(points_needed, len(nearest_points))
            
            if points_to_transfer > 0:
                points_transferred = nearest_points.iloc[:points_to_transfer]
                df_balanced.loc[points_transferred.index, 'ROTA'] = route
        
        iteration += 1
    
    return df_balanced

In [ ]:
# Balanceamento das rotas
data_balanced = iterative_balance_routes(data)

In [ ]:
# Contagem do número de pontos por rota após balanceamento
rota_counts_balanced = data_balanced['ROTA'].value_counts()

In [ ]:
# Classificação das rotas após balanceamento
below_range_balanced = rota_counts_balanced[rota_counts_balanced < 350].count()
within_range_balanced = rota_counts_balanced[(rota_counts_balanced >= 350) & (rota_counts_balanced <= 450)].count()
above_range_balanced = rota_counts_balanced[rota_counts_balanced > 450].count()

In [ ]:
# Exibição dos resultados
print(f"Após balanceamento:")
print(f"Rotas abaixo de 350 pontos: {below_range_balanced}")
print(f"Rotas dentro do intervalo de 350-450 pontos: {within_range_balanced}")
print(f"Rotas acima de 450 pontos: {above_range_balanced}")

In [ ]:
# Salvamento do resultado em CSV
data_balanced.to_csv(f'../cluster/{first_n_clusters}clusters_balanced.csv', index=False)

In [ ]:
# Função para plotar os clusters com destaque para rotas abaixo de 350 pontos
def plot_clusters_sns_highlight_below_350(df, below_routes, num_clusters, cluster_col, title, filename):
    plt.figure(figsize=(10, 6))
    palette = sns.color_palette("hsv", num_clusters)
    mask_below_350 = df['ROTA'].isin(below_routes)
    sns.scatterplot(data=df[~mask_below_350], x='LONGITUDE', y='LATITUDE', hue=cluster_col, palette=palette, s=50, legend=None)
    sns.scatterplot(data=df[mask_below_350], x='LONGITUDE', y='LATITUDE', color='red', s=50, legend=None)
    plt.legend(title='Below 350 Points', loc='upper left', labels=['Below 350 Points'])
    plt.title(title)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.savefig(filename)
    plt.close()

In [ ]:
below_350_routes_balanced = rota_counts_balanced[rota_counts_balanced < 350].index
plot_clusters_sns_highlight_below_350(data_balanced, below_350_routes_balanced, first_n_clusters, 'LEITURISTA', f'{first_n_clusters} Clusters with Below 350 Points Highlighted', f'../cluster/{first_n_clusters}_clusters_highlighted_balanced.png')